# Simple CSTR NMPC
(By David T)

### 0. Introduction
In this example, the control of a simple CSTR reactor is demonstrated with the CAPRESE tools. For this model, a non-isothermal CSTR model is used. The kinetics represent the reaction between thiosulfate and hydrogen peroxide.
\begin{align}
\dfrac{dC_{A}}{dt} &= \dfrac{F}{V} \left(C_{A}^{in} - C_{A} \right) - 2k T_{R}C_{A}^{2} \\
\dfrac{dT_{R}}{dt} &= \dfrac{F}{V} \left(T_{R}^{in} - T_{R} \right) + \dfrac{2 \left(- \Delta H_{R} \right)k T_{R} C_{A}^{2}}{\rho C_{p}} - \dfrac{UA}{V \rho C_{p}} \left(T_{R} - T_{cw} \right) \\
\dfrac{dT_{cw}}{dt} &= \dfrac{F_{cw}}{V_{cw}} \left(T_{cw}^{in} - T_{cw} \right) + \dfrac{2 \left(- \Delta H_{R} \right)k T_{R} C_{A}^{2}}{\rho C_{p}} - \dfrac{UA}{V \rho C_{p}} \left(T_{R} - T_{cw} \right) \\
k \left(T_{R} \right) &= k_{0} \exp \left(\dfrac{-E_{a}}{RT_{R}} \right)
\end{align}
Where $C_{A}$ is the concentration of thiosulfate, $T_{R}$ is the reactor temperature and the cooling water temperature $T_{cw}$.  
For this problem, a simple NMPC strategy will be used, that is a formulation in the form:
 \begin{equation}
     \begin{split}
         \min_{\mathbf{u}_{k}} \quad & \varphi_{N} \left(x_{N|k}\right) +\sum_{i=0}^{N-1} \left[ x_{i|k}^{T}Q_{i}x_{i|k} + u_{i|k}^{T}R_{i}u_{i|k} \right]\\
         \text{s.t.} \quad &x_{l+1|k} = f \left(x_{l|k}, u_{l|k}\right) \\ & x_{0|k} = x \left(k\right) \\
         &x_{l|k} \in \mathbb{X}, \quad l \in \left\lbrace 0,1,...,N-1 \right\rbrace, \quad x_{N|k} \in \mathbb{X}_{f} \\
         &u_{l|k} \in \mathbb{U}, \quad l \in \left\lbrace 0,1,...,N-1 \right\rbrace,\\
     \end{split}
 \end{equation}
where, $x$ and $u$ correspond to states and controls.

### 1. Model set-up

In [ ]:
from pyomo.environ import *
from sample_mods.cstr_rodrigo.cstr_c_nmpc import cstr_rodrigo_dae
from nmpc_mhe.pyomo_dae.NMPCGen_pyDAE import NmpcGen_DAE
from nmpc_mhe.aux.utils import load_iguess
from nmpc_mhe.aux.utils import reconcile_nvars_mequations
import matplotlib.pyplot as plt
import sys, os

To start, the *differential* states of the problem need to be defined in a list of strings.

In [ ]:
states = ["Ca", "T", "Tj"]

In [ ]:
controls = ["u1"]
u_bounds = {"u1": (200, 1000)}

In [ ]:
state_bounds = {"Ca": (0.0, None), "T":(2.0E+02, None), "Tj":(2.0E+02, None)}

In [ ]:
ref_state = {("Ca", (0,)): 0.010}

In [ ]:
mod = cstr_rodrigo_dae(1, 1)

In [ ]:
e = NmpcGen_DAE(mod,
                2,
                states,
                controls,
                var_bounds=state_bounds,
                u_bounds=u_bounds,
                ref_state=ref_state,
                override_solver_check=True)

In [ ]:
e.get_state_vars()
e.load_iguess_steady()
load_iguess(e.SteadyRef, e.PlantSample, 0, 0)
e.create_nmpc()
reconcile_nvars_mequations(e.olnmpc)
e.solve_dyn(e.PlantSample)
e.find_target_ss()
e.PlantSample.display()
e.create_suffixes_nmpc()
e.update_targets_nmpc()
e.compute_QR_nmpc(n=-1)
e.new_weights_olnmpc(1e-04, 1e+06)
e.solve_dyn(e.PlantSample, stop_if_nopt=True)
e.update_state_real()  # update the current state
e.update_soi_sp_nmpc()
e.preparation_phase_nmpc(as_strategy=False, make_prediction=False, plant_state=True)
# e.load_init_state_nmpc(src_kind="state_dict", state_dict="estimated")
e.olnmpc.pprint(filename="new_framework.txt")
stat_nmpc = e.solve_dyn(e.olnmpc, skip_update=False, max_cpu_time=300,
                        jacobian_regularization_value=1e-04, tag="olnmpc",
                        keepsolve=False, wantparams=False)
e.olnmpc.objfun_nmpc.pprint()
e.olnmpc.xmpc_ref_nmpc.display()

e.print_r_nmpc()
e.update_u(e.olnmpc)  #: Get the resulting input for k+1
e.cycleSamPlant(plant_step=True)
e.plant_uinject(e.PlantSample, src_kind="dict", skip_homotopy=True)
# e.noisy_plant_manager(sigma=0.001, action="apply", update_level=True)

In [ ]:
for i in range(0, 500):
    if i in [30 * (j * 2) for j in range(0, 100)]:
        ref_state = {("Ca", (0,)): 0.019}
        e.change_setpoint(ref_state=ref_state, keepsolve=True, wantparams=True, tag="sp")
        e.compute_QR_nmpc(n=-1)
        e.new_weights_olnmpc(1e-04, 1e+06)
    elif i in [30 * (j * 2 + 1) for j in range(0, 100)]:
        ref_state = {("Ca", (0,)): 0.01}
        e.change_setpoint(ref_state=ref_state, keepsolve=True, wantparams=True, tag="sp")
        e.compute_QR_nmpc(n=-1)
        e.new_weights_olnmpc(1e-04, 1e+06)
    e.solve_dyn(e.PlantSample, stop_if_nopt=True)
    e.update_state_real()  # update the current state
    e.update_soi_sp_nmpc()
    e.preparation_phase_nmpc(as_strategy=False, make_prediction=False, plant_state=True)
    stat_nmpc = e.solve_dyn(e.olnmpc, skip_update=False, max_cpu_time=300,
                            jacobian_regularization_value=1e-04, tag="olnmpc",
                            keepsolve=False, wantparams=False)
    if stat_nmpc != 0:
        sys.exit()
    e.print_r_nmpc()
    e.update_u(e.olnmpc)  #: Get the resulting input for k+1
    e.cycleSamPlant(plant_step=True)
    e.plant_uinject(e.PlantSample, src_kind="dict", skip_homotopy=True)
    # sys.exit()
    # e.noisy_plant_manager(sigma=0.01, action="apply", update_level=True)

# print(e.soi_dict[key])
plt.plot(e.soi_dict[("Ca", (0,))])
plt.show()